In [5]:
import time
from datetime import datetime as dt
from biscoint_api_python import Biscoint
from playsound import playsound
import threading
from utils import percent, btcToTrade
from configs import logging
from configs import API_KEY,API_SECRET,AMOUNT,MIN_PERCENT_REQUIRED,BASE_URL,BRL_AMOUNT_TRADE

In [6]:
bsc = Biscoint(API_KEY, API_SECRET)
endpoints_meta = bsc.get_meta()
rate_limit_offer = endpoints_meta['endpoints']['offer']['post']['rateLimit']
sleep_time_offers = ((rate_limit_offer["windowMs"] / rate_limit_offer["maxRequests"]) / 1000)*2
initial_balance = bsc.get_balance()
last_balance = initial_balance
ticker = bsc.get_ticker()
amount_btc_to_trade = btcToTrade(BRL_AMOUNT_TRADE,ticker['askQuoteAmountRef'],ticker['bidBaseAmountRef'])

In [7]:
def async_offer(op:str,amount:str,isQuote:bool):
    response = None
    try:
        response = bsc.get_offer(op,amount,isQuote) 
        request_orders[op] = response
    except Exception as e:
        print(response)
        print("Erro ao rodar thread",e)

In [8]:
# Arbitrage Cycle
cycle_count = 1
percent_record = -1
request_orders = {'buy':None,'sell':None}
while True:
    start_time = dt.now()
    #buy = bsc.get_offer(op='buy',amount=str(amount_btc_to_trade),isQuote=False)
    #sell = bsc.get_offer(op='sell',amount=str(amount_btc_to_trade),isQuote=False)



    thread_buy = threading.Thread(target=async_offer,args=('buy',AMOUNT,False))
    thread_sell = threading.Thread(target=async_offer,args=('sell',AMOUNT,False))
    thread_buy.start()
    time.sleep(0.1)
    thread_sell.start()
    thread_buy.join()
    thread_sell.join()


    calculated_percent = percent(request_orders['buy']['efPrice'],request_orders['sell']['efPrice'])
    logging.info(f"Percent is {calculated_percent} | Cycle {cycle_count}")

    if calculated_percent > percent_record:
        logging.info(f"Percent Record Reached!! : {calculated_percent}")
        percent_record = calculated_percent

    if(calculated_percent >= MIN_PERCENT_REQUIRED):
        logging.info(f"Arbitrage oportunity: buy:{request_orders['buy']['efPrice']}   sell:{request_orders['sell']['efPrice']}")
        playsound('beep.wav')
        #Execute orders
        executed_buy = bsc.confirm_offer(request_orders['buy']['offerId'])
        executed_sell = bsc.confirm_offer(request_orders['sell']['offerId'])
        logging.success(executed_buy)
        logging.success(executed_sell)

        last_balance = bsc.get_balance()
        logging.info(f"New Balance is: {last_balance}")
        break

    end_time = dt.now()
    seconds_elapsed = (end_time - start_time).total_seconds()
    logging.debug(f"Cycle took {seconds_elapsed} seconds")

    #print(f"Took {seconds_elapsed} seconds")
    if cycle_count % 2 == 0:
        try:
            ticker = bsc.get_ticker()
            amount_btc_to_trade = btcToTrade(BRL_AMOUNT_TRADE,ticker['askQuoteAmountRef'],ticker['bidBaseAmountRef'])
        except Exception as e:
            pass

    cycle_count +=1

    # if spead is high, sleep
    if calculated_percent < -0.3:
        time.sleep(sleep_time_offers)
    else:
        time.sleep(1)

Percent is -1.033 | Cycle 1
Percent is -1.21 | Cycle 2
Percent is -1.21 | Cycle 3
None
Erro ao rodar thread 403 Client Error: Forbidden for url: https://api.biscoint.io/v1/offer
Percent is -1.033 | Cycle 4
Percent is -0.697 | Cycle 5
Percent Record Reached!! : -0.697
None
Erro ao rodar thread 403 Client Error: Forbidden for url: https://api.biscoint.io/v1/offer
Percent is -0.697 | Cycle 6
Percent is -0.697 | Cycle 7
Percent is -0.715 | Cycle 8
None
Erro ao rodar thread 403 Client Error: Forbidden for url: https://api.biscoint.io/v1/offer
Percent is -0.715 | Cycle 9
Percent is -0.697 | Cycle 10
None
Erro ao rodar thread 403 Client Error: Forbidden for url: https://api.biscoint.io/v1/offer
None
Erro ao rodar thread 403 Client Error: Forbidden for url: https://api.biscoint.io/v1/offer
Percent is -0.661 | Cycle 11
Percent Record Reached!! : -0.661
Percent is -0.644 | Cycle 12
Percent Record Reached!! : -0.644
Percent is -0.644 | Cycle 13
None
Erro ao rodar thread 403 Client Error: Forbidde

KeyboardInterrupt: 

In [ ]:
#thread_buy = threading.Thread(target=offerFunc,args=('buy',AMOUNT,False,buy))
#thread_sell = threading.Thread(target=offerFunc,args=('sell',AMOUNT,False,sell))
#thread_buy.start()
#thread_sell.start()
#thread_buy.join()
#thread_sell.join()
